In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from collections import Counter


In [2]:
property_df = pd.read_csv("../data/curated/full_data.csv")

In [3]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [4]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [5]:
y = X['cost_text'].tolist()
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1, 
       inplace=True)
X.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category",
          "Offence Count": "int32"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(5, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), ('House', <class 'str'>), (930.0, <class 'numpy.float64'>), (481.0, <class 'numpy.float64'>), (3051.0, <class 'numpy.float64'>), (2779.0, <class 'numpy.float64'>), (8781.0, <class 'numpy.float64'>), ('Warrandyte - Wonga Park', <class 'str'>), ('>50.0', <class 'str'>), (859.0, <class 'numpy.float64'>)]


/tmp/ipykernel_16643/3736008520.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1,


,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count
0,5,3,3,House,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0


In [6]:
type(y[0])

float

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPRegressor(max_iter=5000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)]) # ('reduce_dim', PCA()),



In [ ]:
print('Cross-val standardised features R^2:', np.mean(cross_val_score(pipeline, X, y, cv=10)))
# 36%... not good. might need to perform PCA to improve, and further feature selection.
# can try grid search to tune hyperparams. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train),y_train) # need to perform a train test split here
# need to do balanced split- i.e. one of the bootstrapping methods or something to make sure all suburbs have
# a good amount of representation. 


#print(clf.coefs_)
print('parameter shapes:',[p.shape for p in clf.coefs_])
print('num layers:', clf.n_layers_)
print('R^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) # 34%


In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = clf.predict(preprocessor.transform(X_test))
print(mean_squared_error(y_test, y_pred))

## Some Notes
So we see that we only get an R^2 of 34%. This is before tuning hyperparams, but even then, it is difficult to believe changing hyperparameters will be a significant enough boost. Let us try another method. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

alphas = [np.power(10.0, i) for i in range(-7, 2)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, 
                                                          stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train), y_train)
print('MLP R^2 without tuning:', clf.score(preprocessor.transform(X_test), y_test))

hidden_sizes = [[220], [550, 220]]
#arguments of MLPClassifier and a list of values for them to search and find the best.
param_grid = {'alpha': alphas, 'hidden_layer_sizes':hidden_sizes}

# Make an appropriate scoring function

gs = GridSearchCV(estimator=clf,
                  param_grid=param_grid,
                  scoring='neg_mean_squared_error',
                  cv=3,
                  n_jobs=1, # Somehow verbose will not print if you do multithread
                  verbose=1 # More verbose = More detailed print
                  )

gs.fit(preprocessor.fit_transform(X_train), y_train)
best_params = gs.best_params_
print('best_params', best_params)

clf = MLPRegressor(max_iter=5000, **best_params)
clf.fit(preprocessor.fit_transform(X_train), y_train)
print('R^2 with best params:', clf.score(preprocessor.transform(X_test), y_test))

In [ ]:


import pandas as pd
pd.DataFrame([gs.cv_results_['param_alpha'],
              gs.cv_results_['param_hidden_layer_sizes'],
              gs.cv_results_['mean_test_score'],
              gs.cv_results_['mean_fit_time'],
              gs.cv_results_['rank_test_score']], 
             index=['alpha','Hidden layer size','Mean test score', 'Mean fit time', 'Ranking'])



We see that the accuracy is slightly low, despite our hypter parameter tuning. It is likely that the sheer amount of data is making it difficult to build a good model. We could try dividing things into categories and building seperate models. 

In [8]:
import geopandas as gpd
# sf stands for shape file
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
australia_sf.head(3)
# seperate suburbs that are in Victoria
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

# drop the null location ID values
vic_sf.dropna(inplace=True)
vic_sf.head(1)

/tmp/ipykernel_16643/2689031410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vic_sf.dropna(inplace=True)


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."


In [9]:
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)
cbd_loc = vic_centroid_sf.where(vic_centroid_sf['SA2_NAME21'] == 'Melbourne CBD - North').dropna()['geometry']
cbd_loc

/home/sesshah/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


780    POINT (144.96089 -37.80918)
Name: geometry, dtype: geometry

In [10]:
# function taken from https://stackoverflow.com/a/56661833

from math import radians, cos, sin, asin, sqrt, inf

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# this function converts two floats to a point that is recognised by Shapely
def point_to_dist(row):
    dist = haversine(row["geometry"].y, row["geometry"].x, cbd_loc.y, cbd_loc.x)
    return dist


In [33]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
X3 = vic_centroid_sf.join(X2, how='inner',on='SA2_NAME21')


X3['cbd_dist'] = X3.apply (lambda row: point_to_dist(row), axis=1)
X3["SA2_CODE21"] = X3["SA2_CODE21"].astype(int)

In [34]:
X3.sort_values(by=['cbd_dist'], inplace=True)
X3.set_index('SA2_CODE21', inplace=True)
X3

,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
SA2_CODE21,,,,,,,,,,,,,,,,,,
206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212031557,Cranbourne East - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,11.0254,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.31291 -38.10222),12,47.402490
212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723


In [35]:
property_df['SA2_CODE_2021'] = property_df['SA2_CODE_2021'].astype(int)
property_df

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.000000,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.000000,3,2,1,House,spaciou home east,207021424,145.150927,-37.777456,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.000000,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,-37.775254,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424
3,https://www.domain.com.au/7-davy-court-narre-w...,7 Davy Court Narre Warren South VIC 3805,795.000000,4,2,2,House,bedroom famili home prime locat warren south,212031457,145.318043,-38.061485,767.0,430.0,2350.0,2352.0,10833.0,Narre Warren South - East,>50.0,859.0,212031457
4,https://www.domain.com.au/level-1-301-13-15-la...,"Level 1, 301/13-15 Lake Street Caroline Spring...",793.269231,4,3,2,Apartment / Unit / Flat,endless possibl,213041463,144.744689,-37.731646,813.0,401.0,2277.0,2133.0,16696.0,Caroline Springs,>50.0,859.0,213041463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,https://www.domain.com.au/8-298-nepean-highway...,8/298 Nepean Highway Seaford VIC 3198,365.000000,2,1,2,Apartment / Unit / Flat,easi care lifestyl,214011375,145.124349,-38.128700,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375
10860,https://www.domain.com.au/8-90-nepean-highway-...,8/90 Nepean Highway Seaford VIC 3198,300.000000,2,1,1,Apartment / Unit / Flat,beach side locat,214011375,145.126181,-38.098008,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375
10861,https://www.domain.com.au/86-centenary-street-...,86 Centenary Street Seaford VIC 3198,480.000000,4,2,3,House,solid bedroom home plenti space,214011375,145.158473,-38.110641,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375
10862,https://www.domain.com.au/9-a-ti-tree-crescent...,9 A Ti Tree Crescent Seaford VIC 3198,285.000000,2,1,1,Apartment / Unit / Flat,privat rear courtyard bungalow plu garden bdrm...,214011375,145.156094,-38.111442,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375


In [147]:

#property_df.set_index('SA2_CODE21')
X_dists= property_df.join(X3[['cbd_dist']], how='inner', on='SA2_CODE_2021')

In [148]:
X_dists

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,...,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021,cbd_dist
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
20,https://www.domain.com.au/389a-heidelberg-warr...,389A Heidelberg-Warrandyte Road Warrandyte VIC...,670.0,3,2,2,House,nativ retreat,211021262,145.204065,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
177,https://www.domain.com.au/9-314a-reynolds-road...,9/314A Reynolds Road Donvale VIC 3111,420.0,2,1,1,House,mapl rise independ live year,211021262,145.190657,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.0,3,2,1,House,spaciou home east,207021424,145.150927,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.0,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,https://www.domain.com.au/8-298-nepean-highway...,8/298 Nepean Highway Seaford VIC 3198,365.0,2,1,2,Apartment / Unit / Flat,easi care lifestyl,214011375,145.124349,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10860,https://www.domain.com.au/8-90-nepean-highway-...,8/90 Nepean Highway Seaford VIC 3198,300.0,2,1,1,Apartment / Unit / Flat,beach side locat,214011375,145.126181,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10861,https://www.domain.com.au/86-centenary-street-...,86 Centenary Street Seaford VIC 3198,480.0,4,2,3,House,solid bedroom home plenti space,214011375,145.158473,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832
10862,https://www.domain.com.au/9-a-ti-tree-crescent...,9 A Ti Tree Crescent Seaford VIC 3198,285.0,2,1,1,Apartment / Unit / Flat,privat rear courtyard bungalow plu garden bdrm...,214011375,145.156094,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832


In [46]:
count = 0
BREAK = 2500
li = []
suburb_lists = []
for index, row in X3.iterrows():
    #print(row)
    count += int(row['count'])
    li.append(row["SA2_NAME21"])
    if len(suburb_lists) < 3: 
        if count >= BREAK:
            count = 0
            suburb_lists.append(li)
            li = []
suburb_lists.append(li)

In [49]:
sl = 100
models = []
numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

for i in range(len(suburb_lists)): 
    print(f"\n-- Beginning Ring {i} -- ")
    suburbs = suburb_lists[i]
    test_df = X_dists.where(X_dists['SA2_NAME21'].isin(suburbs)).dropna()

    y = test_df['cost_text'].tolist()
    print("\tnum training instances: ", len(y))
    # note: dropping desc head for now since it is not valuable yet
    # SA2 CODE covered by SA2 Name
    test_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
                  "cost_text", "SA2_NAME21"], axis=1, 
           inplace=True)
    test_df.astype({ 
              "Median_tot_prsnl_inc_weekly":"float", 
              "Median_rent_weekly":"float",
              "Median_tot_fam_inc_weekly":"float", 
              "Median_tot_hhd_inc_weekly":"float", 
              "total_population": "int64",
              "duration_mins": "category", 
              "Bed":"int32", 
              "Bath":"int32", 
              "Park":"int32",
              "property_type":"category",
              #"SA2_NAME21": "category",
              "Offence Count": "int32",
              "cbd_dist": "float"}).dtypes

    X_train, X_test, y_train, y_test = train_test_split(test_df, y, random_state=0, test_size=0.1)
                                                        #stratify=test_df[['SA2_NAME21']])
    clf = MLPRegressor(max_iter=10000,hidden_layer_sizes=[sl])

    clf.fit(preprocessor.fit_transform(X_train),y_train)  

    models.append(clf)
    #print(clf.coefs_)
    print('\tparameter shapes:',[p.shape for p in clf.coefs_])
    print('\tnum layers:', clf.n_layers_)
    print('\tR^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) 
    # 0.36, 0.44, 0.45, 0.51 R^2s- better. 


-- Beginning Ring 0 -- 
	num training instances:  2573
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.233810455273036

-- Beginning Ring 1 -- 
	num training instances:  2498
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.398515433482283

-- Beginning Ring 2 -- 
	num training instances:  2505
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.5049287807390557

-- Beginning Ring 3 -- 
	num training instances:  3264
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.5110137822800109


In [149]:
X_dists.head(1)
categorical_features = ["property_type","duration_mins" ]

In [150]:

cur_avg_rents = X_dists.groupby(['SA2_CODE_2021'])['cost_text'].mean()
cur_avg_rents

SA2_CODE_2021
206011106    502.875000
206011107    453.350000
206011109    487.678571
206011495    550.742706
206011496    548.450704
                ...    
214011372    439.687500
214011373    593.181818
214011374    497.222222
214011375    448.454545
214011376    610.000000
Name: cost_text, Length: 249, dtype: float64

In [151]:
X3

,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
SA2_CODE21,,,,,,,,,,,,,,,,,,
206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212031557,Cranbourne East - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,11.0254,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.31291 -38.10222),12,47.402490
212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723


In [152]:
df_lists = pd.DataFrame([])
for index, row in X3.iterrows():
    cur_sub = row["SA2_NAME21"]
    test_df = X_dists.where(X_dists['SA2_NAME21'] == cur_sub).dropna()
    df_num = pd.DataFrame(test_df[numeric_features].median()).transpose()
    df_cat = pd.DataFrame(test_df[categorical_features].mode())
    df_num['suburb'] = cur_sub
    df_cat['suburb'] = cur_sub
    df_num.set_index('suburb', inplace=True)
    df_cat.set_index('suburb', inplace=True)
    avg_house = df_cat.join(df_num, how='inner')
    if df_lists.empty:
        df_lists = avg_house
    else:
        df_lists = pd.concat([df_lists, avg_house])

In [153]:
df_lists.dropna(inplace=True) # i shouldnt have NAs though. 

In [154]:
df_lists # still 4 more than there should be...

,property_type,duration_mins,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,Bed,Bath,Park,Offence Count,cbd_dist
suburb,,,,,,,,,,,,
Melbourne CBD - North,Apartment / Unit / Flat,10.0,638.0,361.0,1520.0,1089.0,17891.0,2.0,1.0,0.0,1073.0,0.000000
Melbourne CBD - West,Apartment / Unit / Flat,10.0,861.0,376.0,1945.0,1446.0,16491.0,2.0,1.0,0.0,859.0,0.728392
Melbourne CBD - East,Apartment / Unit / Flat,10.0,884.0,375.0,2248.0,1511.0,10533.0,2.0,1.0,0.0,859.0,0.917602
Carlton,Apartment / Unit / Flat,20.0,652.0,365.0,2064.0,1289.0,17080.0,2.0,1.0,0.0,926.0,1.103787
West Melbourne - Residential,Apartment / Unit / Flat,10.0,1047.0,390.0,2397.0,1784.0,7460.0,2.0,1.0,1.0,428.0,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...
Cranbourne East - North,House,40.0,792.0,400.0,1985.0,1931.0,17852.0,4.0,2.0,2.0,859.0,47.402490
Berwick - North,House,30.0,820.0,380.0,2311.0,1885.0,20218.0,3.0,2.0,2.0,1105.0,47.426478
Cranbourne East - South,House,50.0,855.0,401.0,2036.0,1958.0,9482.0,4.0,2.0,2.0,859.0,47.691723


In [156]:
X1 = X_dists.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
                  "cost_text", "SA2_NAME21"], axis=1)
X1.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          #"SA2_NAME21": "category",
          "Offence Count": "int32",
          "cbd_dist": "float"}).dtypes
df_lists = df_lists[X1.columns]

In [157]:
df_lists

,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,duration_mins,Offence Count,cbd_dist
suburb,,,,,,,,,,,,
Melbourne CBD - North,2.0,1.0,0.0,Apartment / Unit / Flat,638.0,361.0,1520.0,1089.0,17891.0,10.0,1073.0,0.000000
Melbourne CBD - West,2.0,1.0,0.0,Apartment / Unit / Flat,861.0,376.0,1945.0,1446.0,16491.0,10.0,859.0,0.728392
Melbourne CBD - East,2.0,1.0,0.0,Apartment / Unit / Flat,884.0,375.0,2248.0,1511.0,10533.0,10.0,859.0,0.917602
Carlton,2.0,1.0,0.0,Apartment / Unit / Flat,652.0,365.0,2064.0,1289.0,17080.0,20.0,926.0,1.103787
West Melbourne - Residential,2.0,1.0,1.0,Apartment / Unit / Flat,1047.0,390.0,2397.0,1784.0,7460.0,10.0,428.0,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...
Cranbourne East - North,4.0,2.0,2.0,House,792.0,400.0,1985.0,1931.0,17852.0,40.0,859.0,47.402490
Berwick - North,3.0,2.0,2.0,House,820.0,380.0,2311.0,1885.0,20218.0,30.0,1105.0,47.426478
Cranbourne East - South,4.0,2.0,2.0,House,855.0,401.0,2036.0,1958.0,9482.0,50.0,859.0,47.691723


In [158]:
X_dists

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,...,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021,cbd_dist
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
20,https://www.domain.com.au/389a-heidelberg-warr...,389A Heidelberg-Warrandyte Road Warrandyte VIC...,670.0,3,2,2,House,nativ retreat,211021262,145.204065,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
177,https://www.domain.com.au/9-314a-reynolds-road...,9/314A Reynolds Road Donvale VIC 3111,420.0,2,1,1,House,mapl rise independ live year,211021262,145.190657,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.0,3,2,1,House,spaciou home east,207021424,145.150927,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.0,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,https://www.domain.com.au/8-298-nepean-highway...,8/298 Nepean Highway Seaford VIC 3198,365.0,2,1,2,Apartment / Unit / Flat,easi care lifestyl,214011375,145.124349,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10860,https://www.domain.com.au/8-90-nepean-highway-...,8/90 Nepean Highway Seaford VIC 3198,300.0,2,1,1,Apartment / Unit / Flat,beach side locat,214011375,145.126181,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10861,https://www.domain.com.au/86-centenary-street-...,86 Centenary Street Seaford VIC 3198,480.0,4,2,3,House,solid bedroom home plenti space,214011375,145.158473,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832
10862,https://www.domain.com.au/9-a-ti-tree-crescent...,9 A Ti Tree Crescent Seaford VIC 3198,285.0,2,1,1,Apartment / Unit / Flat,privat rear courtyard bungalow plu garden bdrm...,214011375,145.156094,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832


In [245]:
def find_ring(index):
    for i in range(len(suburb_lists)):
        suburbs = suburb_lists[i]
        if index in suburbs:
            return i
        

rent_predictions = {}
for index, row in df_lists.iterrows():
    i = find_ring(index)
    rent = models[i].predict(preprocessor.transform(pd.DataFrame(row).transpose()))
    rent_predictions[index] = rent[0]

In [246]:
rent_2022 = rent_predictions

In [247]:
fdf = pd.read_csv("../data/curated/forecast_data.csv")

In [248]:
fdf.drop(['Unnamed: 0'], axis=1, inplace=True)
fdf

,Year,SA2_codes,pred_crime_count,pred_rent,pred_income_prsnl,pred_income_fam,pred_income_hhd,pred_pop
0,2023,201011001,941.366206,394.000000,904.533333,2309.666667,2030.233333,14472.666667
1,2023,201011001,941.366206,394.000000,930.200000,2372.000000,2098.800000,17270.400000
2,2023,201011002,941.366206,325.766667,882.566667,2397.433333,1657.100000,11145.333333
3,2023,201011002,941.366206,334.200000,910.800000,2439.600000,1671.400000,13793.400000
4,2023,201011005,941.366206,349.333333,849.800000,2390.900000,2019.266667,6253.800000
...,...,...,...,...,...,...,...,...
1249,2025,216031416,995.210388,302.000000,869.233333,2194.366667,1755.266667,18543.433333
1250,2025,216031418,995.210388,266.066667,849.300000,2195.733333,1790.966667,3658.333333
1251,2025,216031419,995.210388,272.000000,779.466667,2013.300000,1540.466667,10146.966667
1252,2025,217011421,995.210388,263.000000,755.900000,1811.333333,1371.266667,9403.700000


In [249]:
suburb_map = X3[['SA2_NAME21']]
suburb_map

,SA2_NAME21
SA2_CODE21,
206041504,Melbourne CBD - North
206041505,Melbourne CBD - West
206041503,Melbourne CBD - East
206041117,Carlton
206041510,West Melbourne - Residential
...,...
212031557,Cranbourne East - North
212021293,Berwick - North
212031558,Cranbourne East - South


In [250]:
fdf.rename({'SA2_codes': 'SA2_CODE21'}, axis=1, inplace=True) 
fdf2 = fdf[fdf['SA2_CODE21'].notna()]
fdf2.where(fdf2['Year'] == 2023).dropna()

,Year,SA2_CODE21,pred_crime_count,pred_rent,pred_income_prsnl,pred_income_fam,pred_income_hhd,pred_pop
0,2023.0,201011001.0,941.366206,394.000000,904.533333,2309.666667,2030.233333,14472.666667
1,2023.0,201011001.0,941.366206,394.000000,930.200000,2372.000000,2098.800000,17270.400000
2,2023.0,201011002.0,941.366206,325.766667,882.566667,2397.433333,1657.100000,11145.333333
3,2023.0,201011002.0,941.366206,334.200000,910.800000,2439.600000,1671.400000,13793.400000
4,2023.0,201011005.0,941.366206,349.333333,849.800000,2390.900000,2019.266667,6253.800000
...,...,...,...,...,...,...,...,...
413,2023.0,216031416.0,941.366206,286.000000,821.033333,2057.766667,1646.466667,18029.633333
414,2023.0,216031418.0,941.366206,248.866667,805.900000,2078.533333,1707.566667,3606.333333
415,2023.0,216031419.0,941.366206,256.000000,737.066667,1887.900000,1458.066667,9937.566667
416,2023.0,217011421.0,941.366206,249.000000,719.700000,1723.333333,1302.466667,9323.100000


In [251]:
cfdf = fdf.join(suburb_map, how='inner', on='SA2_CODE21')

In [252]:
cfdf.where(cfdf['Year'] == 2023).dropna()

,Year,SA2_CODE21,pred_crime_count,pred_rent,pred_income_prsnl,pred_income_fam,pred_income_hhd,pred_pop,SA2_NAME21
142,2023.0,206011106.0,288.722222,439.666667,1315.766667,2955.433333,2192.633333,13001.333333,Brunswick East
143,2023.0,206011107.0,1214.622222,382.600000,1044.033333,2682.100000,1865.800000,13252.233333,Brunswick West
144,2023.0,206011109.0,680.855556,442.600000,1008.633333,2940.033333,2407.733333,9066.000000,Pascoe Vale South
146,2023.0,206021110.0,497.811111,398.700000,1148.633333,3297.800000,2238.033333,8167.633333,Alphington - Fairfield
147,2023.0,206021112.0,503.000000,416.933333,1098.700000,2894.866667,2088.766667,17285.366667,Thornbury
...,...,...,...,...,...,...,...,...,...
379,2023.0,214011372.0,579.911111,357.366667,696.966667,1735.833333,1378.100000,17815.300000,Frankston North
380,2023.0,214011373.0,941.366206,441.966667,907.900000,2614.566667,2141.866667,16358.233333,Frankston South
381,2023.0,214011374.0,449.188889,381.166667,906.700000,2386.166667,2036.633333,21665.566667,Langwarrin
382,2023.0,214011375.0,1138.644444,371.033333,852.333333,2197.866667,1583.433333,15460.533333,Seaford (Vic.)


In [253]:
# join fdf to df_lists
cfdf_lists = cfdf.join(df_lists[["property_type", "duration_mins", "Bed", "Bath", "Park", "cbd_dist"]], 
         how="inner", on="SA2_NAME21")
cfdf_lists.rename({
    'pred_crime_count': 'Offence Count',
    'pred_rent': 'Median_rent_weekly',
    'pred_income_prsnl': 'Median_tot_prsnl_inc_weekly',
    'pred_income_fam': 'Median_tot_fam_inc_weekly',
    'pred_income_hhd': 'Median_tot_hhd_inc_weekly',
    'pred_pop': 'total_population'
}, axis=1, inplace=True) 
cfdf_lists = cfdf_lists[['Year', 'SA2_NAME21'] + X1.columns.to_list()]

cfdf_lists

,Year,SA2_NAME21,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,duration_mins,Offence Count,cbd_dist
142,2023,Brunswick East,2.0,1.0,1.0,Apartment / Unit / Flat,1315.766667,439.666667,2955.433333,2192.633333,13001.333333,20.0,288.722222,4.117126
560,2024,Brunswick East,2.0,1.0,1.0,Apartment / Unit / Flat,1366.066667,446.666667,3045.733333,2254.533333,13404.333333,20.0,301.305556,4.117126
978,2025,Brunswick East,2.0,1.0,1.0,Apartment / Unit / Flat,1416.366667,453.666667,3136.033333,2316.433333,13807.333333,20.0,313.888889,4.117126
143,2023,Brunswick West,2.0,1.0,1.0,Apartment / Unit / Flat,1044.033333,382.600000,2682.100000,1865.800000,13252.233333,30.0,1214.622222,4.648073
561,2024,Brunswick West,2.0,1.0,1.0,Apartment / Unit / Flat,1079.133333,391.400000,2773.400000,1921.200000,13328.933333,30.0,1252.355556,4.648073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,2024,Seaford (Vic.),3.0,1.0,1.0,Apartment / Unit / Flat,877.333333,381.133333,2272.466667,1632.733333,15528.133333,20.0,1167.677778,33.402832
1218,2025,Seaford (Vic.),3.0,1.0,1.0,Apartment / Unit / Flat,902.333333,391.233333,2347.066667,1682.033333,15595.733333,20.0,1196.711111,33.402832
383,2023,Skye - Sandhurst,4.0,2.0,1.0,House,942.633333,466.333333,2484.900000,2320.133333,11719.533333,>50.0,941.366206,38.841791
801,2024,Skye - Sandhurst,4.0,2.0,1.0,House,961.533333,475.333333,2545.600000,2374.533333,11967.133333,>50.0,968.288297,38.841791


In [254]:
future_rent_predictions = {}
for year in range(2023, 2026):
    print(year)
    cfdf_yearly = cfdf_lists.where(cfdf['Year'] == year).dropna()
    cfdf_yearly.set_index('SA2_NAME21', inplace=True)
    # predict for each year as above. 
    rent_predictions = {}
    for index, row in cfdf_yearly.iterrows():
        i = find_ring(index)
        rent = models[i].predict(preprocessor.transform(pd.DataFrame(row).transpose()))
        rent_predictions[index] = rent[0]
    future_rent_predictions[year] = rent_predictions

2023
2024
2025


In [255]:
future_rent_predictions


{2023: {'Brunswick East': 931.0386393620789,
  'Brunswick West': 811.2453451688745,
  'Pascoe Vale South': 646.1521680632119,
  'Alphington - Fairfield': 888.8499638515938,
  'Thornbury': 799.1054162740813,
  'Ascot Vale': 837.8330370292873,
  'Flemington': 889.995480575338,
  'Moonee Ponds': 833.4971064231388,
  'Carlton': 694.9416792719553,
  'Parkville': 712.7214352375106,
  'St Kilda East': 964.538919075196,
  'Carlton North - Princes Hill': 1136.9777566307453,
  'Fitzroy North': 1039.4721853592223,
  'Ashburton (Vic.)': 1495.3592642455146,
  'Balwyn': 967.3651088123435,
  'Camberwell': 1456.9053058155469,
  'Hawthorn East': 938.6922863937508,
  'Kew East': 1147.2416324808084,
  'Bulleen': 695.7331701457798,
  'Doncaster': 507.73659214003794,
  'Templestowe': 626.3535699155202,
  'Templestowe Lower': 613.0667911480562,
  'Blackburn': 505.6006617115354,
  'Blackburn South': 524.6907431246073,
  'Box Hill': 422.766318549555,
  'Box Hill North': 464.5196009226427,
  'Burwood (Vic.)': 

In [262]:
future_rent_predictions[2022] = rent_2022
rent_future = pd.DataFrame.from_dict(future_rent_predictions, orient='index')
rent_future.rename({0: 'df'}, axis=1, inplace=True)
rent_future = rent_future.transpose()

In [268]:
rent_future.dropna(inplace=True)
rent_future['growth_2023'] = (rent_future[2023] - rent_future[2022])/rent_future[2022]
rent_future['growth_2024'] = (rent_future[2024] - rent_future[2022])/rent_future[2022]
rent_future['growth_2025'] = (rent_future[2025] - rent_future[2022])/rent_future[2022]

In [269]:
rent_future

,2023,2024,2025,2022,growth_2023,growth_2024,growth_2025
Brunswick East,931.038639,977.835553,1028.448620,880.605993,0.057270,0.110412,0.167887
Brunswick West,811.245345,813.232332,823.665354,812.678658,-0.001764,0.000681,0.013519
Pascoe Vale South,646.152168,677.669270,709.084596,577.783395,0.118329,0.172878,0.227250
Alphington - Fairfield,888.849964,958.383771,1024.505534,794.595761,0.118619,0.206127,0.289342
Thornbury,799.105416,859.083287,927.207654,703.995450,0.135100,0.220297,0.317065
...,...,...,...,...,...,...,...
Frankston North,447.058144,470.282253,485.111486,406.858196,0.098806,0.155887,0.192336
Frankston South,664.746785,673.495587,682.547302,639.999360,0.038668,0.052338,0.066481
Langwarrin,471.445659,483.858524,497.373940,453.402555,0.039795,0.067172,0.096981
Seaford (Vic.),472.357394,481.787161,492.869647,468.070722,0.009158,0.029304,0.052981


In [275]:
rent_future['growth_2023'].nlargest(10)


West Footscray - Tottenham           0.236218
Mentone                              0.187391
Ivanhoe                              0.181649
Hughesdale                           0.179362
Surrey Hills (East) - Mont Albert    0.178426
Caulfield - South                    0.175439
Ashwood - Chadstone                  0.165855
Viewbank - Yallambie                 0.163360
Highett (West) - Cheltenham          0.159965
Heidelberg - Rosanna                 0.159027
Name: growth_2023, dtype: float64

In [276]:
rent_future['growth_2024'].nlargest(10)


West Footscray - Tottenham           0.393275
Ivanhoe                              0.305997
Ashwood - Chadstone                  0.289740
Caulfield - South                    0.281785
Mentone                              0.281372
Surrey Hills (East) - Mont Albert    0.281342
Hughesdale                           0.265220
Highett (West) - Cheltenham          0.264987
Heidelberg - Rosanna                 0.261554
Viewbank - Yallambie                 0.257656
Name: growth_2024, dtype: float64

In [277]:
rent_future['growth_2025'].nlargest(10)

West Footscray - Tottenham           0.617015
Ivanhoe                              0.432671
Ashwood - Chadstone                  0.423023
Caulfield - South                    0.388131
Surrey Hills (East) - Mont Albert    0.384307
Mentone                              0.372318
Highett (West) - Cheltenham          0.370009
Watsonia                             0.366390
Heidelberg - Rosanna                 0.364783
Murrumbeena                          0.359390
Name: growth_2025, dtype: float64